# Introduction

This is a SKKU Business Analytics final project.
The goal of this project is to answer given questions

- Which factor influenced a candidate in getting placed?
- Does percentage matters for one to get placed?
- Which degree specialization is much demanded by corporate?
- Plus through machine learning predict whether student got placed and how much they will earn

As there are many reasons for getting a job or receiving a high salary. Through this data I would like to recognize the biggest influence.
Due to COVID-19 it's hard to get a job now a days. So I think this project will help me what to do to get a nice job and salary.

## Table of contents

1. Library import
2. Exploratory Data Analysis
    - Which factor influenced a candidate in getting placed?
    - Does percentage matters for one to get placed?
    - Which degree specialization is much demanded by corporate?
    - Salary?
3. Factor Selection
4. Verification through ML
    - Binary Classificaion Problem
        : As goal of placement is predicting whether students are placed or not this model can be a good prediction model.
    - Regression Problem
        : There are several factors influencing salary such as degree, experience etc. Therefore regression analysis may find some relationship between certain factor increase is related to higher salary.


## Library import

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import pandas_profiling
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data = pd.read_csv('/kaggle/input/factors-affecting-campus-placement/Placement_Data_Full_Class.csv')

## Exploratory Data Analysis

In [ ]:
#brief data information to recognize data type
"""
sl_no= serial number
gneder
ssc_p= Secondary educaion percentage - 10th grade
ssc_b= Board of education - Central/ Others
hsc_p= Higher secondary Educaion percentage-12th grade
hsc_b= Board of Education - Central/Others
hsc_s= Specialization in Higher Secondary Education- Commerce/Science
degree_p= Degree percentage
degree_t= Field of degree educaion(e.g. Marketing/Science)
workex= Work experience
etest_p= Employability test percentage
specialisation= Post Graduation(MBA)
mba_p= MBA percentage
status= Placed or not placed
Salary= How much
"""
data.info()

In [ ]:
#data report
data.profile_report(title='Campus Placement Data - Report', progress_bar=False)

        1) NAN value at the salary for the people who are not placed 
        2) Value of salary is too high(scaling issue)


In [ ]:
#Replace Nan with 0
data['salary'] = data['salary'].replace(np.nan, 0) 

In [ ]:
#check
data.head(5)

### Which factor influenced a candidate in getting placed?

To figure this out first change string value 'Placed' to 1, 'Not Placed' to 0 

In [ ]:
data_fix = data
data_fix['status'].values[data_fix['status']=='Not Placed'] = 0 
data_fix['status'].values[data_fix['status']=='Placed'] = 1
data_fix.status = data_fix.status.astype('int')

For the percentage variables using heatmap can effectively visualize relationship


In [ ]:
plt.figure(figsize=(14,12))
data2 = data_fix.loc[:,data_fix.columns != 'Id']
sns.heatmap(data2.corr(), linewidth=0.2, cmap="YlGnBu", annot=True)

Through heatmap we can figure that 
1. **degree_p, hsc_p, ssc_p, mba_p **shows high relationship with status(job placement)
    - We can assume that higher educational level gives more chance to get placed
2. **etest_p** shows weak relationship with both status and salary

Next figure out Gender, ssc_b, hsc_b, degree_t, workex, specialization has relationship with placement


In [ ]:
#Gender
sns.countplot("gender", hue="status", data=data)
plt.show()

> 

As there are 139 male and 76 female through calculating ratio to adjust population difference
<p>
There are 39 unplaced male student and 28 unplaced female student
</p>


In [ ]:
male = 39/139
female = 28/76
if male<female:
    print('male has more chance to get placed')
else:
        print('female has more chance to get placed')


In [ ]:
#ssc_b
sns.countplot("ssc_b", hue="status", data=data)
plt.show()

No big difference in ssc board area for get placed. **Doesn't affect placement.**

In [ ]:
#hsc_s
sns.countplot("hsc_s", hue="status", data=data)
plt.show()

hsc specialized in Commerce and science doesn't have difference as they have similar ratio which **doens't affect placement**

In [ ]:
#degree_t
sns.countplot("degree_t", hue="status", data=data)
plt.show()

Degree also **doesn't affect** placement as they show similar ratio around 2:1

In [ ]:
#workexperience
sns.countplot("workex", hue="status", data=data)
plt.show()

Work experience show significant difference that student who worked **get placed more.**

In [ ]:
#specialisation
sns.countplot("specialisation", hue="status", data=data)
plt.show()

more student specializing in finance get more placed than HR. **Specialisation influence placement.**

### Does percentage matters for one to get placed?

Percentatge variables are ssc_p, hssc_p, degree_p, etest_p, mba_p. However as etest doesn't have relationship with placement so I'll drop it. We've alreaddy figured in heat map but let me do it in differenct way.
<p>
To visualize the difference, using barplot will work
</p>

In [ ]:
fig,axes = plt.subplots(3,2, figsize=(20, 20))
sns.barplot(x='status', y='ssc_p', data=data2, ax=axes[0][0])
sns.barplot(x='status', y='hsc_p', data=data2, ax=axes[0][1])
sns.barplot(x='status', y='degree_p', data=data2, ax=axes[1][0])
sns.barplot(x='status', y='mba_p', data=data2, ax=axes[1][1])
fig.delaxes(ax = axes[2][0]) 

Yes again with the heatmap we can recognize percentage variables except etest_p matters with a placement.
<p>
However this boxplot implies that high percentage of certain variable doesn't guarentee getting placed as each bar dosen't show significant difference.
    </p>
    

### Which degree specialization is much demanded by corporate?

We can figure this out through comparing specialisation in marketing vs. finance using countplot

In [ ]:
#specialisation count
sns.countplot(x='specialisation',data=data)
plt.show()

Students specializing in marketing & finance get placed more than markeing & HR

### Salary?

Salary is also a important parameter. Therefore also checkout which factor affect salary.
Salary shows high skewness and outliers(Check line 4).
Through using boxplot and commparing median of the variables, we can determine whether variables affect salary or not.
And for the percentage value use line plot to see whether percentage increase lead to salary increase.
As i have changed NAN value to 0, I have to change the format to NAN again.

In [ ]:
#Replace 0 with NAN
data['salary'] = data['salary'].replace(0, np.nan)

In [ ]:
#gender
plt.figure(figsize =(20,6))
sns.boxplot("salary", "gender", data=data)
plt.show()

Median of Male is higher than female and ew can see more outliers with high salary. Male student gets more salary than female student

In [ ]:
#ssc_b
plt.figure(figsize =(20,6))
sns.boxplot("salary", "ssc_b", data=data)
plt.show()
#ssc_p
sns.lineplot("ssc_p", "salary", hue="ssc_b", data=data)
plt.show()

Median of both central and board student is similar which means there aren't big difference.
<p> Also when we look at the ssc_p, no increase in salary as ssc percentage increase.</p>
Therefore, both ssc_b and ssc_p doesn't affect salary.

In [ ]:
#hsc_b
plt.figure(figsize =(20,6))
sns.boxplot("salary", "hsc_b", data=data)
plt.show()
#hsc_p
sns.lineplot("hsc_p", "salary", hue="hsc_b", data=data)
plt.show()

Like ssc, hsc_b doesn't show big difference. And hsp_p doesnt show increase in salary as hsc percentage increase.
<p> Therefore, hsc doesn't affect salary.

In [ ]:
#degree_p
sns.lineplot("degree_p", "salary", hue="degree_t", data=data)
plt.show()
#degree_t
plt.figure(figsize =(20,6))
sns.boxplot("salary", "degree_t", data=data)
plt.show()

No significant relationship beween degree_p with salary.
<p>And for the degree_t, science and tech student gets more salary than others so it affects salary.


In [ ]:
#workex
plt.figure(figsize =(20,6))
sns.boxplot("salary", "workex", data=data)
plt.show()

Seems like work experience has no affect on salary, but students with work experience can get chance to become outlier(high salary).

In [ ]:
#etest_p
sns.lineplot("etest_p", "salary", data=data)
plt.show()

etest_p don't show any relationship with salary increase.

In [ ]:
#specialisation
plt.figure(figsize =(20,6))
sns.boxplot("salary", "specialisation", data=data)
plt.show()

Marketing and finance shows higher salary and outliers which means specialization affect salary.

In [ ]:
#mba_p
sns.lineplot("mba_p", "salary", data=data)
plt.show()

mba_p don't show any relationship with salary increase.

## Factor selection

Through EDA we can figure out some factors affecting either placement or salary. 

Affecting factors

    - Gender
    - ssc_p
    - hsc_p
    - degree_p
    - degree_t
    - workex
    - specialisation
    - mba_p
<p>
Unaffecting fators are etest_p, ssc_b, hsc_b as they don't affect both placement and salary.
    
    'etest_p','ssc_b','hsc_b'+'sl_no' drop
  

In [ ]:
#Drop data
data.drop(['ssc_b','hsc_b','etest_p','sl_no'], axis=1, inplace=True)

In [ ]:
#Check
data.head(5)

In [ ]:
#Change float data
data["gender"] = data.gender.map({"M":0,"F":1})
data["hsc_s"] = data.hsc_s.map({"Commerce":0,"Science":1,"Arts":2})
data["degree_t"] = data.degree_t.map({"Comm&Mgmt":0,"Sci&Tech":1, "Others":2})
data["workex"] = data.workex.map({"No":0, "Yes":1})
data["specialisation"] = data.specialisation.map({"Mkt&HR":0, "Mkt&Fin":1})

In [ ]:
#check
data.head(5)

## Verification through ML

* Binary Classification Problem : As goal of placement is predicting whether students are placed or not, this model can be a good prediction model.
* Regression Problem : There are several factors influencing salary such as degree, experience etc. Therefore regression analysis may find some relationship between certain factor increase is related to higher salary.


### Binary Classsification Problem for placement

In [ ]:
#make dataset for BCP and Regression
data_placement = data.copy()
data_salary = data.copy()

In [ ]:
#check
data_placement.head(5)

In [ ]:
#check
data_salary.head(5)

### Binary Classification

In [ ]:
#Library import
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report


In [ ]:
# Seperating Variables and Target
x = data_placement[['gender', 'ssc_p', 'hsc_p', 'hsc_s', 'degree_p', 'degree_t', 'workex','specialisation', 'mba_p',]]
y = data_placement['status']

In [ ]:
#Train Test Split(7:3)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)

In [ ]:
#decision tree classifier model
dtree = DecisionTreeClassifier(criterion='entropy')
dtree.fit(x_train, y_train)
y_pred = dtree.predict(x_test)

In [ ]:
#R ^2 
accuracy_score(y_test, y_pred)

In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
#Random Forest Model
random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(x_train, y_train)
y_pred = random_forest.predict(x_test)

In [ ]:
accuracy_score(y_test, y_pred)

In [ ]:
print(classification_report(y_test, y_pred))

RF describe better than decision tree

In [ ]:
#Feature importance
rows = list(x.columns)
imp = pd.DataFrame(np.zeros(6*len(rows)).reshape(2*len(rows), 3))
imp.columns = ["Classifier", "Feature", "Importance"]
#Add Rows
for index in range(0, 2*len(rows), 2):
    imp.iloc[index] = ["DecisionTree", rows[index//2], (100*dtree.feature_importances_[index//2])]
    imp.iloc[index + 1] = ["RandomForest", rows[index//2], (100*random_forest.feature_importances_[index//2])]

In [ ]:
plt.figure(figsize=(15,5))
sns.barplot("Feature", "Importance", hue="Classifier", data=imp)
plt.title("Computed Feature Importance")
plt.show()

hsc_s, degree_t, workex, specialisation are not such an important feature for both Decision tree and Random Forest.


In [ ]:
#Remove hsc_s, degree_t, workex, specialisation
x = data_placement[['gender', 'ssc_p', 'hsc_p', 'degree_p', 'mba_p',]]
y = data_placement['status']

In [ ]:
#Train Test Split(7:3)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)

In [ ]:
#decision tree classifier model
dtree = DecisionTreeClassifier(criterion='entropy')
dtree.fit(x_train, y_train)
y_pred = dtree.predict(x_test)

In [ ]:
#R^2
accuracy_score(y_test, y_pred)

In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
#Random Forest Model
random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(x_train, y_train)
y_pred = random_forest.predict(x_test)

In [ ]:
#R^2
accuracy_score(y_test, y_pred)

In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
#Feature importance
rows = list(x.columns)
imp = pd.DataFrame(np.zeros(6*len(rows)).reshape(2*len(rows), 3))
imp.columns = ["Classifier", "Feature", "Importance"]
#Add Rows
for index in range(0, 2*len(rows), 2):
    imp.iloc[index] = ["DecisionTree", rows[index//2], (100*dtree.feature_importances_[index//2])]
    imp.iloc[index + 1] = ["RandomForest", rows[index//2], (100*random_forest.feature_importances_[index//2])]

In [ ]:
plt.figure(figsize=(15,5))
sns.barplot("Feature", "Importance", hue="Classifier", data=imp)
plt.title("Computed Feature Importance")
plt.show()

We can conclude that ssc_p, Secondary educaion percentage is the most important feature for placement.
Follwed by hsc_p, degree_p, mba_p, gender.

### Regression Problem for salary

In [ ]:
#library import
from sklearn.preprocessing import MinMaxScaler # for scaling Salary
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from sklearn.metrics import mean_absolute_error, r2_score

In [ ]:
#drop salary NaN 
data_salary.dropna(inplace=True)
#drop Status
data_salary.drop("status", axis=1, inplace=True)

In [ ]:
#check
data_salary.head(5)

In [ ]:
#Seperating Depencent and Independent Vaiiables
y = data_salary["salary"] 
x = data_salary.drop("salary", axis=1)
column_names = x.columns.values

In [ ]:
#scale salary 0-1
x_scaled = MinMaxScaler().fit_transform(x)

In [ ]:
#PDF of Salary
sns.kdeplot(y)
plt.show()

In [ ]:
# Salary outlier removal over 400,000 as most of boxplot marks outlier over 400,000
x_scaled = x_scaled[y < 400000]
y = y[y < 400000]

In [ ]:
#check
sns.kdeplot(y)
plt.show()

The significance of regression model can be determined by
R square, p value and Residual sum of square.
After running a regression analysis remove least significant variable and do it again till only one left.
This is a model proposed by Atish Adhikari
[Refernece](http://www.kaggle.com/atishadhikari/placement-dataanalysis-classification-regression)
<p>
    I'll try to determine the significance using p value with giving α=0.05
<p>
H0 is there is no difference between two samples of R squared scores.  
<p> If the calculated p-value turns out to be less than 0.05, the null hypothesis is considered to be false. And if the value is greater than 0.05, the null hypothesis is considered to be true.

For a feature, a small p-value indicates that it is unlikely we will observe a relationship between the feature and salary variables due to chance.

Therefore,compute the P-values. Eliminate highest p-value until p-values of all features reach below 0.05

In [ ]:
#Converting to DF for as  column names gives readibility
x_scaled = pd.DataFrame(x_scaled, columns=column_names)
y = y.values

# We must add a constants 1s for intercept before doing Linear Regression with statsmodel
x_scaled = sm.add_constant(x_scaled)
x_scaled.head()
#Constants 1 added for intercept term

In [ ]:
# All
model = sm.OLS(y, x_scaled)
results = model.fit()
results.summary()

In [ ]:
# Drop biggest p value= ssc_p
x_scaled = x_scaled.drop('ssc_p', axis=1)
model = sm.OLS(y, x_scaled)
results = model.fit()
results.summary()

In [ ]:
# Drop biggest p value= degree_p
x_scaled = x_scaled.drop('degree_p', axis=1)
model = sm.OLS(y, x_scaled)
results = model.fit()
results.summary()

In [ ]:
# Drop biggest p value= specialisation
x_scaled = x_scaled.drop('specialisation', axis=1)
model = sm.OLS(y, x_scaled)
results = model.fit()
results.summary()

In [ ]:
# Drop biggest p value= hsc_p
x_scaled = x_scaled.drop('hsc_p', axis=1)
model = sm.OLS(y, x_scaled)
results = model.fit()
results.summary()

In [ ]:
# Drop biggest p value= workex
x_scaled = x_scaled.drop('workex', axis=1)
model = sm.OLS(y, x_scaled)
results = model.fit()
results.summary()

In [ ]:
# Drop biggest p value= hsc_s
x_scaled = x_scaled.drop('hsc_s', axis=1)
model = sm.OLS(y, x_scaled)
results = model.fit()
results.summary()

In [ ]:
# Drop biggest p value= degree_t
x_scaled = x_scaled.drop('degree_t', axis=1)
model = sm.OLS(y, x_scaled)
results = model.fit()
results.summary()

Finally all factors p value is under 0.05.
<p>
Through the regression we can assume that gender and mba_p is affecting salary most.
    

### Conclusion

For the placement both RF and Decision tree shows high R^2 score(over 0.8) which well explains whether students get placed or not. And ssc_p is the most importnat issue for getting placed.
<p>
    For the salary with regression R^2 of the model is really low around 0.1, which this model has weak power to explain which factor affects salary. But at least gender and mba_p affects salary within all factors.

In [ ]:
print('Thank you')